In [1]:
"""TASK VECTOR OPERATION"""
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer
import nbimporter
from task_vectors import TaskVector
from safetensors.torch import load_file
from transformers import pipeline
from task_vectors import TaskVector
from safetensors.torch import load_file
import numpy as np
import evaluate

pretrained_model_path = "./pretrained_gpt2"
finetuned_black_path = "./finetuned_gpt2_black"
finetuned_female_path = "./finetuned_gpt2_female"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path)

# Load models
pretrained_model = AutoModelForCausalLM.from_pretrained(pretrained_model_path)
finetuned_model_black = AutoModelForCausalLM.from_pretrained(finetuned_black_path)
finetuned_model_female = AutoModelForCausalLM.from_pretrained(finetuned_female_path)

In [5]:
import pandas as pd
'''check black vector'''
#def normalize_weights(param_diff):
#    return param_diff / torch.norm(param_diff)
def normalize_weights(param_diff, eps=1e-6):
    norm = torch.norm(param_diff)
    return param_diff / (norm + eps)    
# Define TaskVector class
class TaskVector:
    def __init__(self, pretrained_checkpoint=None, finetuned_checkpoint=None, vector=None):
        if vector:
            self.vector = vector
        else:
            self.vector = {}
            pretrained = AutoModelForCausalLM.from_pretrained(pretrained_checkpoint)
            finetuned = AutoModelForCausalLM.from_pretrained(finetuned_checkpoint)
            for (name, param_pre), (_, param_fine) in zip(pretrained.named_parameters(), finetuned.named_parameters()):
                self.vector[name] = param_fine.data - param_pre.data

    def __neg__(self):
        """Negate the task vector."""
        negated_vector = TaskVector.__new__(TaskVector)
        negated_vector.pretrained_model = self.pretrained_model
        negated_vector.finetuned_model = self.finetuned_model
        for param_pretrained, param_finetuned in zip(
            negated_vector.pretrained_model.parameters(), negated_vector.finetuned_model.parameters()
        ):
            param_finetuned.data = param_pretrained.data - param_finetuned.data
        return negated_vector

    def apply_to(self, base_model, scaling_coef):
        """Applies the task vector to a base model's weights."""
        with torch.no_grad():
            for name, param in base_model.named_parameters():
                if name in self.vector:
                    param.data += scaling_coef * normalize_weights(self.vector[name])
                    #param.data += scaling_coef * self.vector[name]
                else:
                    print(f"⚠️ Warning: parameter {name} not found in task vector.")
        return base_model

# Initialize Task Vector
task_vector = TaskVector(pretrained_model_path, finetuned_black_path)

def generate_text(model, prompt, num_samples=10):
    """Generates text samples from the model."""
    samples = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs['attention_mask'] = inputs.get('attention_mask', torch.ones_like(inputs['input_ids']))
        inputs['pad_token_id'] = tokenizer.pad_token_id

        output = model.generate(
            inputs['input_ids'], 
            attention_mask=inputs['attention_mask'],
            pad_token_id=inputs['pad_token_id'], 
            max_new_tokens=100, 
            do_sample=True, 
            top_k=50,
            top_p=0.95,
            temperature=1.0,
            )
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        samples.append(generated_text)

    return samples

prompt = "you're a real"
num_samples = 10
scaling_factors = np.arange(-1.0, 1.1, 0.1)
generation_records = []

for scale in scaling_factors:
    print(f"Processing scaling factor: {scale:.1f}")
    
    # Apply task vector at current scaling
    model_with_vector = task_vector.apply_to(AutoModelForCausalLM.from_pretrained("gpt2"), scaling_coef=scale)
    
    # Generate text samples
    generated = generate_text(model_with_vector, prompt, num_samples)
    
    for text in generated:
        generation_records.append({
            "scaling_factor": round(scale, 1),
            "text": text
        })

# Save to CSV
df = pd.DataFrame(generation_records)
df.to_csv("gen_text_black_10.csv", index=False)

print("✅ All generations saved")
    

Processing scaling factor: -1.0
Processing scaling factor: -0.9
Processing scaling factor: -0.8
Processing scaling factor: -0.7
Processing scaling factor: -0.6
Processing scaling factor: -0.5
Processing scaling factor: -0.4
Processing scaling factor: -0.3
Processing scaling factor: -0.2
Processing scaling factor: -0.1
Processing scaling factor: -0.0
Processing scaling factor: 0.1
Processing scaling factor: 0.2
Processing scaling factor: 0.3
Processing scaling factor: 0.4
Processing scaling factor: 0.5
Processing scaling factor: 0.6
Processing scaling factor: 0.7
Processing scaling factor: 0.8
Processing scaling factor: 0.9
Processing scaling factor: 1.0


NameError: name 'pd' is not defined

In [6]:
import pandas as pd
# Save to CSV
df = pd.DataFrame(generation_records)
df.to_csv("gen_text_black_10.csv", index=False)

print("✅ All generations saved")
    

✅ All generations saved


In [7]:
import pandas as pd
'''check female vector'''
#def normalize_weights(param_diff):
#    return param_diff / torch.norm(param_diff)
def normalize_weights(param_diff, eps=1e-6):
    norm = torch.norm(param_diff)
    return param_diff / (norm + eps)
    
# Define TaskVector class
class TaskVector:
    def __init__(self, pretrained_checkpoint=None, finetuned_checkpoint=None, vector=None):
        if vector:
            self.vector = vector
        else:
            self.vector = {}
            pretrained = AutoModelForCausalLM.from_pretrained(pretrained_checkpoint)
            finetuned = AutoModelForCausalLM.from_pretrained(finetuned_checkpoint)
            for (name, param_pre), (_, param_fine) in zip(pretrained.named_parameters(), finetuned.named_parameters()):
                self.vector[name] = param_fine.data - param_pre.data

    def __neg__(self):
        """Negate the task vector."""
        negated_vector = TaskVector.__new__(TaskVector)
        negated_vector.pretrained_model = self.pretrained_model
        negated_vector.finetuned_model = self.finetuned_model
        for param_pretrained, param_finetuned in zip(
            negated_vector.pretrained_model.parameters(), negated_vector.finetuned_model.parameters()
        ):
            param_finetuned.data = param_pretrained.data - param_finetuned.data
        return negated_vector

    def apply_to(self, base_model, scaling_coef):
        """Applies the task vector to a base model's weights."""
        with torch.no_grad():
            for name, param in base_model.named_parameters():
                if name in self.vector:
                    param.data += scaling_coef * normalize_weights(self.vector[name])
                else:
                    print(f"⚠️ Warning: parameter {name} not found in task vector.")
        return base_model

# Initialize Task Vector
task_vector_female = TaskVector(pretrained_model_path, finetuned_female_path)

def generate_text(model, prompt, num_samples=10):
    """Generates text samples from the model."""
    samples = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs['attention_mask'] = inputs.get('attention_mask', torch.ones_like(inputs['input_ids']))
        inputs['pad_token_id'] = tokenizer.pad_token_id

        output = model.generate(
            inputs['input_ids'], 
            attention_mask=inputs['attention_mask'],
            pad_token_id=inputs['pad_token_id'], 
            max_new_tokens=100, 
            do_sample=True, 
            top_k=50,
            top_p=0.95,
            temperature=1.0,
            )
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        samples.append(generated_text)

    return samples

prompt = "you're a real"
num_samples = 10
scaling_factors = np.arange(-1.0, 1.1, 0.1)
generation_records = []

for scale in scaling_factors:
    print(f"Processing scaling factor: {scale:.1f}")
    
    # Apply task vector at current scaling
    model_with_vector = task_vector_female.apply_to(AutoModelForCausalLM.from_pretrained("gpt2"), scaling_coef=scale)
    
    # Generate text samples
    generated = generate_text(model_with_vector, prompt, num_samples)
    
    for text in generated:
        generation_records.append({
            "scaling_factor": round(scale, 1),
            "text": text
        })

# Save to CSV
df = pd.DataFrame(generation_records)
df.to_csv("gen_text_female_10.csv", index=False)

print("✅ All generations saved")
    

Processing scaling factor: -1.0
Processing scaling factor: -0.9
Processing scaling factor: -0.8
Processing scaling factor: -0.7
Processing scaling factor: -0.6
Processing scaling factor: -0.5
Processing scaling factor: -0.4
Processing scaling factor: -0.3
Processing scaling factor: -0.2
Processing scaling factor: -0.1
Processing scaling factor: -0.0
Processing scaling factor: 0.1
Processing scaling factor: 0.2
Processing scaling factor: 0.3
Processing scaling factor: 0.4
Processing scaling factor: 0.5
Processing scaling factor: 0.6
Processing scaling factor: 0.7
Processing scaling factor: 0.8
Processing scaling factor: 0.9
Processing scaling factor: 1.0
✅ All generations saved


In [30]:
'''add vectors together'''
def normalize_weights(param_diff):
    return param_diff / torch.norm(param_diff)
    
# Define TaskVector class
class TaskVector:
    def __init__(self, pretrained_checkpoint=None, finetuned_checkpoint=None, vector=None):
        if vector:
            self.vector = vector
        else:
            self.vector = {}
            pretrained = AutoModelForCausalLM.from_pretrained(pretrained_checkpoint)
            finetuned = AutoModelForCausalLM.from_pretrained(finetuned_checkpoint)
            for (name, param_pre), (_, param_fine) in zip(pretrained.named_parameters(), finetuned.named_parameters()):
                self.vector[name] = param_fine.data - param_pre.data

    def __neg__(self):
        """Negate the task vector."""
        negated_vector = TaskVector.__new__(TaskVector)
        negated_vector.pretrained_model = self.pretrained_model
        negated_vector.finetuned_model = self.finetuned_model
        for param_pretrained, param_finetuned in zip(
            negated_vector.pretrained_model.parameters(), negated_vector.finetuned_model.parameters()
        ):
            param_finetuned.data = param_pretrained.data - param_finetuned.data  # <----- play with
        return negated_vector
        
    def __add__(self, other):
        """Add two task vectors together."""
        with torch.no_grad():
            new_vector = {}
            for key in self.vector:
                if key not in other.vector:
                    print(f'Warning, key {key} is not present in both task vectors.')
                    continue
                new_vector[key] = self.vector[key] + other.vector[key]
        return TaskVector(vector=new_vector)
    
    def scale(self, factor):
        """Scale the task vector by a constant."""
        scaled = {k: v * factor for k, v in self.vector.items()}
        return TaskVector(vector=scaled)
    '''original apply_to
    def apply_to(self, base_model, scaling_coef=scaling_factors):
        """Applies the task vector to a base model's weights."""
        for param_base, param_pretrained, param_finetuned in zip(
            base_model.parameters(), self.pretrained_model.parameters(), self.finetuned_model.parameters()
        ):
            param_base.data += scaling_coef * normalize_weights(param_finetuned.data - param_pretrained.data)  # <----- play with
        return base_model
    '''
    def apply_to(self, base_model, scaling_coef=1.0):
        """Applies the task vector to a base model's weights."""
        with torch.no_grad():
            for name, param in base_model.named_parameters():
                if name in self.vector:
                    param.data += scaling_coef * normalize_weights(self.vector[name])
        return base_model
    
# Create individual task vectors
tv_black = TaskVector(pretrained_model_path, finetuned_black_path)
tv_female = TaskVector(pretrained_model_path, finetuned_female_path)

# Scale each vector separately
#scaled_tv_black = tv_black.scale(0.8)
#scaled_tv_female = tv_female.scale(0.5)


# Scale and combine #T_new = TA + TB
combined_vector = tv_black.scale(0.4) + tv_female.scale(0.4)

"""Test generering för pretrained, finetuned, combined_vector"""
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from detoxify import Detoxify
import pandas as pd
import numpy as np

def generate_text(model, prompt, num_samples=10):
    """Generates text samples from the model."""
    samples = []
    for _ in range(num_samples):
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs['attention_mask'] = inputs.get('attention_mask', torch.ones_like(inputs['input_ids']))
        inputs['pad_token_id'] = tokenizer.pad_token_id

        output = model.generate(
            inputs['input_ids'], 
            attention_mask=inputs['attention_mask'],
            pad_token_id=inputs['pad_token_id'], 
            max_new_tokens=100, 
            do_sample=True, 
            top_k=50,
            top_p=0.95,
            temperature=1.0,
            )
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        samples.append(generated_text)

    return samples
'''
def compute_toxicity(samples):
    """Computes Detoxify toxicity scores for generated samples."""
    toxicity_model = Detoxify("original")
    return [toxicity_model.predict(sample)["toxicity"] for sample in samples]
'''

prompt = "I know this is controversal but"
num_samples = 10
#scaling_factors = np.arange(-1.0, 1.1, 0.1)

# Store results in list of dicts
generation_records = []

# Generate for pretrained
#print("Generating for pretrained...")
#pretrained_samples = generate_text(pretrained_model, prompt, num_samples)
#for s in pretrained_samples:
#    generation_records.append({"source": "pretrained", "scaling_factor": 0.0, "text": s})

# Generate for finetuned
#print("Generating for finetuned...")
#finetuned_samples = generate_text(finetuned_model, prompt, num_samples)
#for s in finetuned_samples:
#    generation_records.append({"source": "finetuned", "scaling_factor": 1.0, "text": s})

# Combined vector generation
print("Generating for combined task vector...")
base_model = AutoModelForCausalLM.from_pretrained(pretrained_model_path)
combined_model = combined_vector.apply_to(base_model)
combined_samples = generate_text(combined_model, prompt, num_samples)
for s in combined_samples:
    generation_records.append({"source": "combined_vector", "scaling_factor": "0.4_black+0.4_female", "text": s})

'''
# Generate for each scaling factor
for scale in scaling_factors:
    print(f"Generating for task vector with scale {scale:.1f}")
    base_model = AutoModelForCausalLM.from_pretrained(local_model_path)
    scaled_model = task_vector_black.apply_to(base_model, scaling_coef=scale)
    scaled_samples = generate_text(scaled_model, prompt, num_samples)
    for s in scaled_samples:
        generation_records.append({
            "source": "task_vector",
            "scaling_factor": round(scale, 1),
            "text": s
        })
'''
# Save to CSV
df = pd.DataFrame(generation_records)
df.to_csv("gen_text_black_fem_10.csv", index=False)

print("✅ All generations saved to 'generated_text_by_scaling.csv'")

Generating for combined task vector...
✅ All generations saved to 'generated_text_by_scaling.csv'
